# Team members

### Felipe Ramirez (331471)

Bachelor en Microtechnique

### Paul-Antoine Croux

### Seif Labib (326481)

### Nathann Morand (296190)
Electronicien CFC.

Work as a system administrator beside EPFL.

Socially not very aware.

Can code in Python.

# Environment
Our work environment is quite diverse, integrating various tools and platforms. We primarily use PyCharm and VSCode as IDEs, with Git for version control. Some team members prefer GitKraken or GitHub’s web interface. To minimize merging issues, we predefined a class skeleton to structure each team member's work. Function names and return types were also established beforehand to reduce incompatibilities and simplify testing.

As for the robot’s environment, we designed a scenario where the robot acts as an ambulance. Its mission is to reach the site of an accident by navigating roads and avoiding cars (obstacles) along its path. The onboard camera functions as a “GPS,” providing absolute positioning, though it is subject to temporary outages and cannot detect dynamic obstacles. The robot and target locations are marked using two ArUco markers in the form of QR codes, while a checkerboard is used to calibrate the camera and determine the scale in millimeters. The robot is controlled asynchronously via Python.



# Design choice

### Architecture overview (from README.md)
Main loop :
- get the Camera position of the map and robot and the target (car crash, ciff)
- get the map layout (road line) (Camera)
- get the position from the robot encoder
- use kalmann to improve robot position
- update robot position
- use path planning to find best way (A*)
- update robot movement instruction

Fast loop :
- react to obstacle and avoid,
- follow track given by path planning

Pathplanning :
- use the occupancy matrix to compute the fastest route

ComputerVision :
- get the image dimmension + calibrate the camera.
- get the robot + goal position
- get the occupancy map

Robot :
deal with the hardware + realtime
- give estimated position
- allow the update of the estimated position
- get a list of waypoint and follow them.

sensorfusion :
- merge the data from the camera + encoder to improve the accuracy.

### Robot movement + Local obstacle avoindance -> Seif

python class for robot movement.
this is the fast loop.
It run in its own thread and handle to robot movement  + communication and local obstacle avoidance.

The robot movement is based on a few key elements, namely the position of the robot, being an attribute of the robot that can be accessed and modified with a setter and a getter, then the waypoints, also being an attribute than can be accessed and modified with a setter and a getter, which expresses the desired list of waypoints (positions) that we want the robot to follow in order to get to the target (pathplanner.py, the slower thread) and will follow them, using a PID controller for the forward speed, and a simple P controller for the direction of the robot. Both control steps are done separately as when testing I found that combining them in a single step causes a lot of overshoot (in the cases where the angle difference is so little and the distance between the robot and the target (waypoint) is so large, or the other way around if the angle difference is so large while the distance is so little), this overshoot being very slowly treated by the PID controller, I decided to separate both controls and it was so practical.
For the PID controller, the Ki, Kp, and Kd coefficients were experimentally decided, with the constraint of having the whole system behaviour as stable as it can be, but also as fast or as 'real-time' as we can, because at the end of the day the robot simulates an ambulance car.

In the update() function, where everything is computed and the speeds of the motors are set, we set the position of the robot using two key values, the kalman filter value from the slower loop, as well the estimated position using the wheel encoder of the robot.

When it comes to the local avoidance algorithm, the algorithm used was basic and was based upon the IR sensor values.
I only used the front sensors, but within these 5 sensor values, we had an internal hierarchy, meaning that not every sensor has the same importance, this hierarchy was simply implemented using different threshold for the different sensors.
This hierarchy was based on the fact that the robot can't crash into an obstacle in front of it, but can navigate normally when an obstacle is next to it, even if close. So the highest sensors in our robot were the front center left and front center right, to which we implemented a change of directions, and then a simple forward speed then it rechecks if the obstacle is avoided so it re-corrects its direction and follows the planned path, or if not keep avoiding the obstacle and moving forward and then rechecking whether the obstacle was avoided or not and repeat the loop until the obstacle is completely avoided.
This hierarchy was put into place because the case where the obstacle is so thin it can only be detected by one of the sensors, namely the front center one, without the front center right or left sensors was so extreme and rare. That's why the priority was given to the front center right and left ones, as they provide more info than the front center one, a last case where the robot was completely "surrounded" by obstacles was implemented just in case, for it to be a complete local avoidance algorithm.
For the direction change, the angle change, or rather the direction change was around 1 radian, so around 57 degrees, which at first was a random choice, but then proved to be a good one, as it is not so fast/brusque that it produces overshoot neither is it so small so that we would have to stay in the loop for so long.
While for the simple forward speed, it was the same idea, a random not so high speed was set and then found reasonable seeing the behaviour of the robot, although it could've been set higher (we still have an acceptable marge of error).

Finally, for the testing, the main idea was to test the PID controller, as well as the obstacle avoidance algorithm, so the testing was just a simple square that the robot has to form, where I used different obstacles as my hand, water bottles.


also make the thymio blink in blue like an ambulance.

```
class RobotMovement:
    def __init__(self):
        # Initialize motors, sensors, and encoders
        self.waypoints = [np.array([0, 0, 0])] # list of np.array
        self.position = np.array([0, 0, 0])
        self.speed = np.array([0, 0, 0])

    def set_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # same format as get_waypoints
        # update the global list of waypoint to follow

    def get_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # return the global list of waypoint to follow minus the one that where reached aldready
        waypoints = [
            np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians) clockwise rotation from x axis
            np.array([200, 250, np.pi / 3]),  # x=200mm, y=250mm, direction=60° (π/3 radians)
            np.array([300, 350, np.pi / 2])   # x=300mm, y=350mm, direction=90° (π/2 radians)
        ]
        return waypoints

    def get_position(self):# Beware, this class is inside a thread you need to use a mutex !!!
        # Return estimated position from encoders
        # same format as for set_position

    def set_position(self, kalman_position):# Beware, this class is inside a thread you need to use a mutex !!!
        # Update the robot’s position based on Kalman filter results
        return np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians)

    def get_speed(self): # there is no need to set the speed from kalman, the motor know better
        # Return the speed+angular speed from encoders
        # Beware, this class is inside a thread you need to use a mutex !!!
        return np.array([10, 15, np.pi / 60]),  # x=10mm/s, y=15mm/s, direction=3°/s (π/60 radians/second)

    def update(self):# this function is called every 10 ms for you to do what you have to
                     # (avoid obstacle, go to the nearest waypoint)
        return None
```
### Computer vision and global positionning + occupancy grid -> Felipe
This class sets up a computer vision system using OpenCV, with the main goal of generating an occupancy grid and determining the position of our robot and goal. This grid represent the environment as occupied and free area, based on images captured from a camera or loaded from a file. The grid will then be used by other modules/class in the project for tasks like pathplanning.

The VisionSystem class offers a way to capture images, allowing the use of either a real-time camera or image files, which is especially useful for testing. It uses ArUco markers to detect two key elements: the robot and the goal. Using these marker, the class can locate these objects in the environment, calculating their positions in millimeters and their orientations.

Camera calibration is included to improve the accuracy of measurements. It uses a checkerboard pattern to adjust the camera parameters and calculate the size of a pixel in millimeter. This ensures that image coordinates can be converted into real-world distances.

Finally, the main method, which generates the occupancy grid, processes the image to identify fixed obstacles in the environment. It converts the image to grayscale, applies a filter to reduce noise, and binarizes the image to create a clear map of occupied and free areas. The resulting grid will then be used by other project modules.

To provide more clarity, let me explain some key aspects of our code:

For the detection of the ArUco markers we use the function cv2.aruco.detectMarkers() from OpenCV. This function is a encapsulation several processes into a single call. We found a good explanation on internet about what process is used. It's divide in two main steps : https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html
- "Detection of marker candidates. In this step the image is analyzed in order to find square shapes that are candidates to be markers. It begins with an adaptive thresholding to segment the markers, then contours are extracted from the thresholded image and those that are not convex or do not approximate to a square shape are discarded. Some extra filtering is also applied (removing contours that are too small or too big, removing contours too close to each other, etc)."

- "After the candidate detection, it is necessary to determine if they are actually markers by analyzing their inner codification. This step starts by extracting the marker bits of each marker. To do so, a perspective transformation is first applied to obtain the marker in its canonical form. Then, the canonical image is thresholded using Otsu to separate white and black bits. The image is divided into different cells according to the marker size and the border size. Then the number of black or white pixels in each cell is counted to determine if it is a white or a black bit. Finally, the bits are analyzed to determine if the marker belongs to the specific dictionary. Error correction techniques are employed when necessary."

For each detected marker, we calculated its center to simplify the next steps, as this provides the precise position of the objects (Robot and goal). Next, we verified whether the desired ID is present in the previously created matrix to ensure the object was detected. Finally, we constructed a vector, vector_x, to determine the angle between the marker and the horizontal axis. Like this we return a np.array() with the position x-y and the orientation of our object. 

For the calibration, we used a checkboard which provided the parameters needed to achieve the best calibration for our camera. In the first time we 


### Path Planning -> Paul-Antoine
bref tu raconte comment ça marche et pour mettre des extratide code voici la sintaxe :
```
class PathPlanner:
    #
    def get_waypoints(occupancy_grid, start, target):
        # A* algorithm to calculate path
        # Return a list of waypoints to follow, read the class of work package 1 + 2 to have the specific of the format. (robot + VisionSystem)
        start = np.array([100, 150, np.pi / 6]) # both start and target are defined as numpy array with the first component
        target = np.array([100, 150, np.pi / 6]) # being the x axis, second is y, and third is the angle relative to x axis in radian
                                                 # clockwise rotation
        waypoints = [
            np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians)
            np.array([200, 250, np.pi / 3]),  # x=200mm, y=250mm, direction=60° (π/3 radians)
            np.array([300, 350, np.pi / 2])   # x=300mm, y=350mm, direction=90° (π/2 radians)
        ]
        return waypoints
```

### sensor fusion and test script -> Nathann
I tried at first to implement a kalman filter that worked relatively well in "static" condition (linear acceleration or constant speed)
![Kalman Early Test](testData/kalmanEarlyTest.jpg)
but since the robot is changing course from time to time I did not like this solution.
instead I implemented a particule filter that was more robust to the constantly changing course.
![Particule Filter](testData/particuleFilter.jpg)
The particle filter class implement the filter itself (along each DOF of the robot so X position, Y position and theta) and offer the following function :

init: Create the particle filter with a specified number of particles and the max range (we assume that the robot will not travel outside an 10x10 meter arena nor will do more than one full turn on itself before rewinding)

Predict: Move particles by a distance obtained by mutliplying the current speed by the elapsed time and adding some noise. (the noise amplitude is determined empirically to be of the same magnitude of the noise from the sensor)

Update: this function compute the weight (higher mean more likely to reflect the true position of the robot) of each particle compared to the value returned from the sensor. (when both the camera + robot odometry are avaible, do the average between the two)

Resample: This function act as a lotery where a higher weight mean more chance of winning. it take all the "winning" particle (some particle win multiple time) and replace the population of N particle with N new particle made of the winner by the number of win of each winner. this is a zero sum game so that the particle number stay constant.

Estimate: compute the mean position of the particles to find a close approximation of the true position of the robot.

The sensor fusion class keep track of the 3 instance of the particle filter (for each DOF) and check that the value are available for each sensor (if yes, compute the average of the two sensor, if not only use the data from one sensor.

I also made some test script to help the others to verify that the code was working. They take the form of a
"if __name__ == "__main__":" in each file to test the file individually with the help of a visualisation tool.

The Visualisation library offer function function to :

update_plot : This take into argument the posisiton of the robot from the robot and the camera + the goal position and the estimated position. It also take the list of waypoints to reach the target and the occupancy grid and the raw image from openCV and make a nice chat using matplotlib :
![visualisation](testData/visualisation.jpg)

plot_robot_grid : this is similar to update_plot but more crude as it does not update the plot dynamically and is used for static test of the pathfinding algoritm and the computer vision test.

Finally I wrote the main.py and test.py. The "fast" loop freqencythat run in its own thread and control the robot is limited by the asynchronous nature of the robot as it take more than 500 ms query the robot and update its value.
The slow loop run at 0.2 Hz as matplotlib it quite slow to update the visualisation. But since the robot is quite slow this is okay.


# Demo
For a full demo you can run main.pyby following the bash command shown after. Albeit its not functional due to the robot not moving correctly.

If you only wish to simulate the working of the system withouth the robot you can use test.py

Finally, each module can be individually run and tested. sensorfusion generate a plot that show how it behave to a simulated input from two noisy sensor. pathplanner load an occupancy grid and show a visualisation of the path found. robot movement was supposed to follow a set of waypoint and vision system show the occupancygrid from a picture and the goal + robot found using their marker.

to setup your environement clone and cd in this directory and run the following command (tested on ubuntu 20)

In [ ]:
python3 -m venv .venv # setup a virtual environement to prevent library issue
source .venv/bin/activate  # or .venv\Scripts\activate on Windows
pip install -r requirements.txt # install our required library
python3 robot.py # python3 test.py

# Conclusion

Even tho our project is not fully functional some module work well and have been toroughly tested. We are especially happy with the approach of writing the class and define ahead how the module will interact to try to catch has many bug ahead as possible.